## Initializing functions

In [ ]:
""" RUN THIS CELL FIRST """

import csv
import pickle
import numpy as np
import random as rnd
from scipy.optimize import curve_fit
from scipy.integrate import odeint
import matplotlib as mpl
import pylab as plt
import os

# sets a pretty font for figures:
# mpl.rcParams['mathtext.fontset'] = 'stix'
# mpl.rcParams['font.family'] = 'STIXGeneral'

# set the current working directory (ensure it includes the data files):
os.chdir('<place_your_path_here>')   # sets the cwd as '<place_your_path_here>'


# Useful functions -------------------------------------------------------------------------------#

""" identify when and how high the peak of a given population is """
def peak(array):
    array_max = max(array)
    index = list(array).index(array_max)
    return (t[index], array_max)  # <-- 't' is globally defined as the time to inegrate the model across

""" area gives the area under the curve given by array, a0 is an initial value to shift the area by"""
def area(array, steps, a0=0):
    a = np.zeros(len(steps))
    i = 0
    for r in steps:
        a[i] = sum(array[:r])
        i += 1
    return a * D/t_steps + a0 * np.ones(len(a))

""" cumul_30 gives the area under the curve given by array up to the 30th day """
def cumul_30(array, a0=0):
    a = np.zeros(30)
    i = 0
    for r in range(0, 30*dt, dt):
        a[i] = sum(array[:r])
        i += 1
    return (a * D/t_steps  + a0 * np.ones(len(a)))[-1]

""" Ro returns the reproductive ratio for our model, given the parameter values """
def Ro(ba, bi, bw, sa, si, eps, nuu, mui, muu, omeg, kk, pp):
    exit_rates = (eps + muu) * (mui + nuu) * (muu + omeg)
    zeta = eps * (ba * (mui + nuu) + bi * (1 - pp) * omeg) / exit_rates
    eta = 4 * eps * bw * (sa * (mui + nuu) + si * (1 - pp) * omeg) / (kk * exit_rates)
    return (zeta + np.sqrt(zeta**2 + eta))/2

""" returns total number infected over time iINCLUDING the environment """
def I_WAIT(days, ba, bi, bw, sa, si, eps):
    Y = odeint(dXdt, y0=X0, t=t, args=(ba, bi, bw, sa, si, eps))
    omega = 1/(inc_pd - eps)
    new_inf = (1 - p) * omega * Y[:, 2]
    return daily_cases(new_inf)

""" returns total number infected over time WITHOUT the environment """
def I_noWAIT(days, ba, bi, eps):
    Y = odeint(dXdt, y0=X0, t=t, args=(ba, bi, 0, 0, 0, eps))
    omega = 1/(inc_pd - eps)
    new_inf = (1 - p) * omega * Y[:, 2]
    return daily_cases(new_inf)

""" the Akaike Information Criterion is used to asses the validity of fits below """
def AIC(thry, data, k_theta):  # <-- k_theta is the number of fitting params
    """ taken from "AIC under the Framework of Least Squares Estimation"
        by H.T. Banks and Michele L. Joyner
    """
    N = len(data)
    sMLE = np.sqrt(sum((thry - data)**2) / N)  # <-- Maximum-likelihood estimate of std.
    a = - N * np.log(np.sqrt(2 * np.pi) * sMLE)
    b = - sum((thry - data)**2)/(2 * sMLE**2)
    aic = 2 * (k_theta - (a+b))  # <-- the Akaike Information Criterion (a+b is the log-likelihood)
    return aic

def daily_cases(array, num_days=30):
    day_steps = range(0, (num_days+1)*dt, dt)
    cases = np.zeros(num_days)
    for i in range(num_days):
        a, b = day_steps[i], day_steps[i+1]
        cases[i] = sum(array[a:b]) 
    return cases * D/t_steps

%matplotlib inline

#-------------------------------------------------------------------------------------------------#

""" Things to note:

    - In this notebook, it is more convenient to set epsilon = avg. number of days in the E-box (as 
    opposed to the reciprocal of this quantity as in the paper)
    
    - For ease of notation, we use A and I in place of I_A and I_S in the paper; accordingly, where the 
    subscripts _A and _S are used in the paper we use _A and_I here on the betas, sigmas, and on mu
    
    - Many of the cells in this notebook include the average parameter values to make them self-contained. 
    Except for needing to run the "Initializing functions" cell, most cells can be run independently of others.
    Where there is a need to run a prior cell, this is explcitly stated at the top of the cell. 
    
"""

# Model parameters -------------------------------------------------------------------------------#

D = 2*365               # number of days (SETS THE TIMESCALE OF THE PARAMETERS)
t_steps = 10**4         # number of integration steps (increase for better time resolution)
inc_pd = 5.5            # expected incubation period
dt = round(t_steps/D)           # number of integration steps per day
t = np.linspace(0, D, t_steps)  # Initializing the time over which to integrate (be mindful of the limits)

""" average parmeters """
# FITTED -----------------------------#
# bA = 0.5497938413115383
# bI = 0.4909463148261044
# bW = 0.031100446700197397
# sA = 3.403914785083266
# sI = 13.492166134223032
# epsilon = 2.478385725999245
# omega = 0.33094892640811974
#-------------------------------------#

# FIXED ------------------------------#
nu = 0.03053968253968254  # = (1 - 0.038)/(4.5 * 7): assumes 3-6 weeks of recovery, 3.8% death rate
mu = 1/(80.3 * 365)       # 80.3 years is the avg. life expectancy of the 17 countries sampled (weighted by pop.)
muI = mu + 0.038/(3.5*7)  # assumes 3.8% death rate, and 3-4 weeks before death once symptomatic
k = 0.6486486486486486    # = 1/(avg. survival time)
p = 0.956                 # = fraction that move along the "mild" symptomatic route
#-------------------------------------#

# Initial conditions (population averaged across 17 countries sampled):
X0 = [57.05 * 10**6, 5*13.3, 13.3, 13.3, 0, 0.01]

#-------------------------------------------------------------------------------------------------#

# The model --------------------------------------------------------------------------------------#

def dXdt(X, t, bA, bI, bW, sA, sI, epsilon): # This function gives the derivatives of the appropriate populations @ t=0
    
    S = X[0]
    E = X[1]
    A = X[2]
    I = X[3]
    R = X[4]
    W = X[5]

    N = S + E + A + I + R
    L = (bA * A + bI * I)/N + bW * W
    
    omega = 1/(inc_pd - epsilon)
    
    dSdt = mu * (N - S) - L * S
    dEdt = L * S - (1/epsilon + mu) * E
    dAdt = (1/epsilon) * E - (omega + mu) * A
    dIdt = (1 - p) * omega * A - (nu + muI) * I
    dRdt = p * omega * A + nu * I - mu * R
    dWdt = (sA * A + sI * I) * (1 - W)/N - k * W

    return np.array([dSdt, dEdt, dAdt, dIdt, dRdt, dWdt])

#-------------------------------------------------------------------------------------------------#


## Plotting the model dynamics

In [ ]:
""" Average parameters """
bA = 0.5497938413115383
bI = 0.4909463148261044
bW = 0.031100446700197397
sA = 3.403914785083266
sI = 13.492166134223032
epsilon = 2.478385725999245
omega = 0.33094892640811974
mu = 3.411863047817261*10**(-5)
muI = 0.0015851390386414379
nu = 0.03053968253968254
k = 0.6486486486486486
p = 0.956

t = np.linspace(0, D, t_steps) 
Y = odeint(dXdt, y0=X0, t=t, args=(bA, bI, bW, sA, sI, epsilon))

S = Y[:, 0]
E = Y[:, 1]
A = Y[:, 2]
I = Y[:, 3]
R = Y[:, 4]
W = Y[:, 5]

""" plotting the SEAIR-W model """

xlim = 365

fig, ax = plt.subplots(1, 2, figsize = [11, 4])

plt.subplots_adjust(hspace = 0.4)

ax[0].plot(t, S, label = 'S', linewidth = 3, c = 'C0')
ax[0].plot(t, E, label = 'E', linewidth = 3, c = 'C1')
ax[0].plot(t, A, label = 'A', linewidth = 3, c = 'C5')
ax[0].plot(t, I, label = 'I', linewidth = 3, c = 'C2')
ax[0].plot(t, R, label = 'R', linewidth = 3, c = 'C3')
ax[0].plot(t, S + E + A + I + R, label = 'total', linewidth = 3, c = 'C4')
ax[1].plot(t, W, linewidth = 3)

ax[0].legend(fontsize = 12)
ax[0].set_title('Infection dynamics', fontsize = 20)
ax[0].set_xlim(0, xlim)
ax[0].set_ylabel('number of people', fontsize = 15)
ax[0].set_xlabel('days', fontsize = 15)
ax[0].ticklabel_format(axis = 'y', style = 'sci', scilimits = (0, 1))
ax[1].set_xlabel('days', fontsize = 15)
ax[1].set_xlim(0, xlim)
ax[1].set_title('Fraction of environment infected', fontsize = 20)

for i in range(2):
    ax[i].yaxis.get_offset_text().set_fontsize(15)
    ax[i].tick_params(axis = 'both', direction = 'in', top = True, right = True, labelsize = 15)

plt.tight_layout()


In [ ]:
""" save the infected population arrays as csv files """

""" pick a population to save to csv! """
population, array = 'S', S
# population, array = 'E', E
# population, array = 'A', A
# population, array = 'I', I
# population, array = 'W', W
# population, array = 'R', R
# population, array = 'SEAIR', S+E+A+I+R
            
np.savetxt(f"avg pops_{population}.csv", array, delimiter=",")

## Surface analysis

In [ ]:
""" In this cell and the following, we consider how various model aspects vary with k (these aspects include
    the population values, the R0-value, the time to the symptomatic peak, the value of said peak, 
    and the total number of symptomatically infected people after 30 days). 
    
    In this cell, we consider how dynamics vary for specific surfaces (values of k).
    In the next cell, we look at how other model aspects vary continuously with a range of k values.
"""

""" average parameters """
bA = 0.5497938413115383
bI = 0.4909463148261044
bW = 0.031100446700197397
sA = 3.403914785083266
sI = 13.492166134223032
epsilon = 2.478385725999245
omega = 0.33094892640811974
mu = 3.411863047817261*10**(-5)
muI = 0.0015851390386414379
nu = 0.03053968253968254
k = 0.6486486486486486
p = 0.956

X0 = [57.05 * 10**6, 5*13.3, 13.3, 13.3, 0, 0.01]

""" this version of the model makes k as the only free parameter: """
def dXdt(X, t, k): # This function gives the derivatives of the appropriate populations @ t=0
    S = X[0]
    E = X[1]
    A = X[2]
    I = X[3]
    R = X[4]
    W = X[5]

    N = S + E + A + I + R
    L = (bA * A + bI * I)/N + bW * W
    
    omega = 1/(inc_pd - epsilon)
    
    dSdt = mu * (N - S) - L * S
    dEdt = L * S - (1/epsilon + mu) * E
    dAdt = (1/epsilon) * E - (omega + mu) * A
    dIdt = (1 - p) * omega * A - (nu + muI) * I
    dRdt = p * omega * A + nu * I - mu * R
    dWdt = (sA * A + sI * I) * (1 - W)/N - k * W

    return np.array([dSdt, dEdt, dAdt, dIdt, dRdt, dWdt])


""" comment-in a surface! """
k, surface = 1/(4/24), 'Copper'
k, surface = 1/(24/24), 'Cardboard'
k, surface = 1/(48/24), 'Stainless Steel'
k, surface = 1/(72/24), 'Plastic'

t = np.linspace(0, D, t_steps)  # Initializing the time over which to integrate (be mindful of the limits)
Y = odeint(dXdt, y0=X0, t=t, args=(k,))

S = Y[:, 0]
E = Y[:, 1]
A = Y[:, 2]
I = Y[:, 3]
R = Y[:, 4]
W = Y[:, 5]

xlim = 365  # <-- set the xlimit on the graph

fig, ax = plt.subplots(1, 2, figsize = [11, 4])

plt.subplots_adjust(hspace = 0.4)

# ax[0].plot(t, S, label = 'S', linewidth = 3, c = 'C0')
ax[0].plot(t, E, label = 'E', linewidth = 3, c = 'C1')
ax[0].plot(t, A, label = 'A', linewidth = 3, c = 'C5')
ax[0].plot(t, I, label = 'I', linewidth = 3, c = 'C2')
# ax[0].plot(t, R, label = 'R', linewidth = 3, c = 'C3')
# ax[0].plot(t, S + E + A + I + R, label = 'total', linewidth = 3, c = 'C4')
ax[1].plot(t, W, linewidth = 3)

# ax[0].scatter(range(21), infected_in_china)

""" model aspects to include in the figures as text """
r0 = round(Ro(bA, bI, bW, sA, sI, 1/epsilon, nu, muI, mu, omega, k, p), 2)
t_max = round(peak(I)[0], 1)
I_max = round(peak(I)[1], -1)
cumul30 = round(cumul_30((1 - p) * omega * A, a0 = X0[3]), -1)

""" plotting the figures """
ax[0].legend(fontsize = 12)
ax[0].set_title('Infection among people', fontsize = 20)
ax[0].set_xlim(0, xlim)
ax[0].set_ylim(0, 4 * 10**6)
ax[0].set_ylabel('number of people', fontsize = 15)
ax[0].set_xlabel('days', fontsize = 15)
ax[0].ticklabel_format(axis = 'y', style = 'sci', scilimits = (0, 1))
ax[0].text(0.4, 0.85, f"$R_0$ = {r0}", transform = ax[0].transAxes, fontsize = 15)
ax[0].text(0.4, 0.75, f"$t_{{max}}$ = {t_max}", transform = ax[0].transAxes, fontsize = 15)
ax[0].text(0.4, 0.65, f"$I_{{max}}$ = {I_max}", transform = ax[0].transAxes, fontsize = 15)
ax[0].text(0.4, 0.55, f"30 day total = {cumul30}", transform = ax[0].transAxes, fontsize = 15)
ax[1].set_xlabel('days', fontsize = 15)
ax[1].set_xlim(0, xlim)
ax[1].set_ylim(0, 1)
ax[1].set_title('Fraction of environment infected', fontsize = 20)
ax[1].text(0.6, 0.8, f'{surface}', transform = ax[1].transAxes, fontsize = 20, 
            bbox=dict(facecolor = 'none', edgecolor = 'k', boxstyle='round'))

for i in range(2):
    ax[i].yaxis.get_offset_text().set_fontsize(15)
    ax[i].tick_params(axis = 'both', direction = 'in', top = True, right = True, labelsize = 15)

plt.tight_layout()

""" printing some of the model details """
tot_cases_30 = cumul_30((1 - p) * omega * A, a0 = X0[3])
tot_death_30 = cumul_30(muI * I)

print(surface)
print('')
print('peak is ', peak(I))
print('total infected after 30 days is', tot_cases_30)
print('R0 is', r0)
print('number of deaths is', tot_death_30)
print('death ratio is', tot_death_30/tot_cases_30)

In [ ]:
""" save the infected population arrays as csv files """

""" pick a population to save to csv! """
population, array = 'E', E
# population, array = 'A', A
# population, array = 'I', I
# population, array = 'W', W
            
np.savetxt(f"{surface}_{population}.csv", array, delimiter=",")

In [ ]:
""" we can also sweep across many values of k """

""" average parameters """
bA = 0.5497938413115383
bI = 0.4909463148261044
bW = 0.031100446700197397
sA = 3.403914785083266
sI = 13.492166134223032
epsilon = 2.478385725999245
omega = 0.33094892640811974
mu = 3.411863047817261*10**(-5)
muI = 0.0015851390386414379
nu = 0.03053968253968254
k = 0.6486486486486486
p = 0.956

X0 = [57.05 * 10**6, 5*13.3, 13.3, 13.3, 0, 0.01]

def dXdt(X, t, k): # This function gives the derivatives of the appropriate populations @ t=0
    S = X[0]
    E = X[1]
    A = X[2]
    I = X[3]
    R = X[4]
    W = X[5]

    N = S + E + A + I + R
    L = (bA * A + bI * I)/N + bW * W
    
    omega = 1/(inc_pd - epsilon)
    
    dSdt = mu * (N - S) - L * S
    dEdt = L * S - (1/epsilon + mu) * E
    dAdt = (1/epsilon) * E - (omega + mu) * A
    dIdt = (1 - p) * omega * A - (nu + muI) * I
    dRdt = p * omega * A + nu * I - mu * R
    dWdt = (sA * A + sI * I) * (1 - W)/N - k * W

    return np.array([dSdt, dEdt, dAdt, dIdt, dRdt, dWdt])

""" the base values (models values for k = 0.6486): """
Y = odeint(dXdt, y0=X0, t=t, args=(k,))

base_R0 = Ro(bA, bI, bW, sA, sI, epsilon, nu, muI, mu, omega, k, p)
base_time = peak(Y[:, 3])[0]
base_peak = peak(Y[:, 3])[1]
base_30 = cumul_30((1 - p) * omega * Y[:, 2], a0 = X0[3])

base_vals = [base_R0, base_time, base_peak, base_30]

""" space to sweep across k """
ks = np.linspace(1/(72/24), 1/(1/24), 100)

""" running the k-sweep """
R0s = []
peak_times = []
peak_vals = []
inf_30s = []

for k in ks:
    
    Y = odeint(dXdt, y0=X0, t=t, args=(k,))
    
    R0s.append(Ro(bA, bI, bW, sA, sI, epsilon, nu, muI, mu, omega, k, p))
    peak_times.append(peak(Y[:, 3])[0])
    peak_vals.append(peak(Y[:, 3])[1])
    inf_30s.append(cumul_30((1 - p) * omega * Y[:, 2], a0 = X0[3]))


""" plotting the k-sweep """
titles = ['$\mathscr{R}_0$', '$t_{max}$', '$I_{max}$', '30 day total'] 
ylabels = ['', 'Days', 'Number of people', 'Number of people']
low_high = [(0.7, 1.25), (0.6, 2), (0.7, 1.15), (-1, 3.5)]
top_bottom = [(0, -1), (-1, 0), (0, -1), (0 , -1)]
shifts = [(0.98, 0.98), (0.98, 0.98), (0.99, 0.99), (0.94, -6)]
label = ['A', 'C', 'D', 'B']
    
i = 0
for to_plot in [R0s, peak_times, peak_vals, inf_30s]:
    
    title = titles[i]
    ylabel = ylabels[i]
    base = base_vals[i]
    low, high = low_high[i]
    a, b = top_bottom[i]
    shifta, shiftb = shifts[i]
    
    up = round(100*(to_plot[a] - base)/base)
    down = round(100*(base - to_plot[b])/base)
    
    fig, ax = plt.subplots(figsize = [6, 4])
    
    ax.plot(1/ks, to_plot, linewidth = 3)
    ax.plot([1/ks[0], 1/ks[-1]], [base] * 2, dashes = [3, 6], c = 'k')
    ax.plot([1/ks[0], 1/ks[-1]], [to_plot[a]] * 2, dashes = [3, 6], c = 'r')
    ax.plot([1/ks[0], 1/ks[-1]], [to_plot[b]] * 2, dashes = [3, 6], c = 'r')
    ax.text(1.05/ks[0], shifta*to_plot[a], f'$\leftarrow$ +{up}%', fontsize = 16)
    ax.text(1.05/ks[0], shiftb*to_plot[b], f'$\leftarrow$ -{down}%', fontsize = 16)
    ax.text(-0.2, 1.1, label[i], fontsize = 30, fontweight = 'bold', transform = ax.transAxes)
    ax.ticklabel_format(axis = 'y', style = 'sci', scilimits = (0, 1))
    ax.yaxis.get_offset_text().set_fontsize(15)
    ax.set_ylim(base * low, base * high)
    ax.set_ylabel(f'{ylabel}', fontsize = 20)
    ax.set_title(f'{title}', fontsize = 25)
    ax.set_xlabel('$1/k$ (days)', fontsize = 20)
    ax.tick_params(axis = 'both', direction = 'in', top = True, right = True, labelsize = 15)
    
""" save to png with this line """
#     plt.savefig(f'{title} surface sweep', dpi = 300, bbox_inches = 'tight')
    
    i += 1
    

## Fitting the model to each country

In [ ]:
""" RUN THIS CELL BEFORE RUNNING THE SEIR-W OR SEIR CELL BELOW """

""" after running either the SEIR-W cell or the SEIR cell, run the "Averaging 
    the parameters across the countries" cell to get the parameter values, given in the order: 
    bA, bI, bW, sA, sI, epsilon for SEIR-W and bA, bI, epsilon for SEIR
"""

""" We fit bA, bI, bW, sA, sI, & epsilon for the SEIR-W model, and bA, bI, & epsilon for the SEIR model """

""" countries sampled """
countries = ['Australia',
            'Austria',
            'Canada',
            'China',
            'Denmark',
            'France',
            'Germany',
            'Iran',
            'Italy',
            'Netherlands',
            'Norway',
            'South Korea',
            'Spain',
            'Sweden',
            'Switzerland',
            'United Kingdom',
            'United States']


# Collecting the populations ------------------------------#

file_name = 'country population.txt'

with open(file_name, 'r') as f:  
    reader = csv.reader(f)
    pops = list(reader)    

country_labels = [x[1] for x in pops]
    
country_pop = {}
for country in countries:
    country_index = country_labels.index(country)
    country_pop[country] = int(pops[country_index][-1])
    
""" we set the population of China to that of the approximate pop. of the Hubei province """
country_pop['China'] = 60*10**6

#----------------------------------------------------------#

# Collecting the total infected cases data ----------------#

""" retrieving the total counts data """
file_name1 = 'euroCDC CoV-19 data.txt'

with open(file_name1, 'r') as f:  
    reader = csv.reader(f)
    total_cases = list(reader)     
    
#----------------------------------------------------------#

# Collecting the daily new cases --------------------------#

""" retrieving the daily cases data """
file_name2 = 'euro CDC daily cases.txt'  

with open(file_name2, 'r') as f:  
    reader = csv.reader(f)
    new_cases = list(reader)    
    
#----------------------------------------------------------#


### SEIR-W (with environment)

In [ ]:
%%time

""" RUN THE "Fitting the model to each country" CELL BEFORE RUNNING THIS CELL """

""" RUN THIS FOR THE MODEL INCLUDING THE ENVIRONMENT """

country_vals = {} # <-- to contain the (aic, popt, pcov) for each country
I0s = {}          # <-- to contain the initial count of susceptible folks

for country in country_pop:
    
    print(country)  # <-- this helps keep track of the progress
    
    #------------------------------------------------------#
    """ getting the country data """
    
    country_index = total_cases[0].index(country)
    
    country_total_cases = [int(total_cases[i][country_index]) for i in range(1, len(total_cases))]
    country_new_cases = [int(new_cases[i][country_index]) for i in range(1, len(new_cases))]
      
    start = 0
    for counts in country_total_cases:
        if counts >= 10:  # <-- we consider the first day greater or equal to 10 counts
            break
        start += 1
    
    new_cases_data = country_new_cases[start: start+30]  # <-- 30 days of data
    #------------------------------------------------------#

    I0 = country_total_cases[start]              # <-- initial symptomatic value
    I0s[country] = I0                   # <-- saved per country

    days = range(len(new_cases_data))
    
    # we use country-specific initial conditions:
    X0 = [country_pop[country], 5*I0, I0, I0, 0, 0.01]

    # running the fitting algorithm ------------------------------------------------------#
    init_params = [1.5] * 6  # <-- [bA, bI, bW, sA, sI, epsilon]
    bounds = ([0] * 6, [100, 100, 100, 100, 100, inc_pd*0.99])
    popt, pcov = curve_fit(I_WAIT, days, new_cases_data, p0=init_params, bounds=bounds)
    bA, bI, bW, sA, sI, epsilon = popt
    omega = 1/(inc_pd - epsilon)
    Y = odeint(dXdt, y0=X0, t=t, args=(bA, bI, bW, sA, sI, epsilon))
    new_cases_thry = daily_cases((1 - p) * omega * Y[:, 2])
    aic = AIC(new_cases_thry, new_cases_data, len(init_params))
    #-------------------------------------------------------------------------------------#
    
    country_vals[country] = (aic, popt, pcov)
    

### SEIR (no environment)

In [ ]:
%%time

""" RUN THE "Fitting the model to each country" CELL BEFORE RUNNING THIS CELL """

""" RUN THIS FOR THE MODEL WITHOUT THE ENVIRONMENT """

country_vals_noWAIT = {} # <-- to contain the (aic, popt, pcov) for each country
I0s = {}          # <-- to contain the initial count of susceptible folks

for country in country_pop:
    
    print(country)
    
    #------------------------------------------------------#
    country_index = total_cases[0].index(country)
    
    country_total_cases = [int(total_cases[i][country_index]) for i in range(1, len(total_cases))]
    country_new_cases = [int(new_cases[i][country_index]) for i in range(1, len(new_cases))]
      
    start = 0
    for counts in country_total_cases:
        if counts >= 10:  # <-- we consider the first day greater or equal to 10 counts
            break
        start += 1
    
    new_cases_data = country_new_cases[start: start+30]  # <-- 30 days of data
    #------------------------------------------------------#

    I0 = country_total_cases[start]              # <-- initial symptomatic value
    
    I0s[country] = I0                   # <-- saved per country

    days = range(len(new_cases_data))

    X0 = [country_pop[country], 5*I0, I0, I0, 0, 0.01]

    init_params = [1.5]*3  # <-- [bA, bI, epsilon]
    bounds = ([0] * 3, [100, 100, inc_pd*0.99])
    popt, pcov = curve_fit(I_noWAIT, days, new_cases_data, p0=init_params, bounds=bounds)
    bA, bI, epsilon = popt
    omega = 1/(inc_pd - epsilon)
    Y = odeint(dXdt, y0=X0, t=t, args=(bA, bI, 0, 0, 0, epsilon))
    new_cases_thry = daily_cases((1 - p) * omega * Y[:, 2])
    aic = AIC(new_cases_thry, new_cases_data, len(init_params))

    country_vals_noWAIT[country] = (aic, popt, pcov)
    

## Averaging the paramters across the countries

In [ ]:
""" RUN TO GET THE FITTED PARAMETERS FOR THE SEIR-W MODEL """
un_wghtd_avg = sum([country_vals[country][1] for country in country_pop])/len(country_pop)
un_wghtd_sd = np.std([country_vals[country][1] for country in country_pop], axis = 0)

print('means')
for each in un_wghtd_avg:
    print(each)
print('')
print('st. dev.')
for each in un_wghtd_sd:
    print(each)

In [ ]:
""" RUN TO GET THE FITTED PARAMETERS FOR THE SEIR MODEL """
un_wghtd_avg = sum([country_vals_noWAIT[country][1] for country in country_pop])/len(country_pop)
un_wghtd_sd = np.std([country_vals_noWAIT[country][1] for country in country_pop], axis = 0)

print('means')
for each in un_wghtd_avg:
    print(each)
print('')
print('st. dev.')
for each in un_wghtd_sd:
    print(each)

### Plotting the fits for a specific country

In [ ]:
""" RUN THIS CELL BEFORE RUNNING INDIVIDUAL FITS BELOW """

""" We fit: bA, bI, bW, sA, sI, epsilon """

""" countries sampled """
countries = ['Australia',
            'Austria',
            'Canada',
            'China',
            'Denmark',
            'France',
            'Germany',
            'Iran',
            'Italy',
            'Netherlands',
            'Norway',
            'South Korea',
            'Spain',
            'Sweden',
            'Switzerland',
            'United Kingdom',
            'United States']

# Collecting the populations ------------------------------#

file_name = 'country population.txt'

with open(file_name, 'r') as f:  
    reader = csv.reader(f)
    pops = list(reader)    

country_labels = [x[1] for x in pops]
    
country_pop = {}
for country in countries:
    country_index = country_labels.index(country)
    country_pop[country] = int(pops[country_index][-1])
    
""" we set the population of China to that of the approximate pop. of the Hubei province """
country_pop['China'] = 60*10**6

#----------------------------------------------------------#

# Collecting the total infected cases data ----------------#

""" retrieving the total counts data """
file_name1 = 'euroCDC CoV-19 data.txt'

with open(file_name1, 'r') as f:  
    reader = csv.reader(f)
    total_cases = list(reader)     
    
#----------------------------------------------------------#

# Collecting the daily new cases --------------------------#

""" retrieving the daily cases data """
file_name2 = 'euro CDC daily cases.txt'  

with open(file_name2, 'r') as f:  
    reader = csv.reader(f)
    new_cases = list(reader)    
    
#----------------------------------------------------------#


In [ ]:
""" RUN THE CELL ABOVE THIS ONE TO INITIALIZE THE DATA """

#------------------------------------------------------#

""" pick a country! """

country = 'United States'
# label = ['G', 'H']

country_index = total_cases[0].index(country)
    
country_total_cases = [int(total_cases[i][country_index]) for i in range(1, len(total_cases))]
country_new_cases = [int(new_cases[i][country_index]) for i in range(1, len(new_cases))]

start = 0
for counts in country_total_cases:
    if counts >= 10:  # <-- we consider the first day greater or equal to 10 counts
        break
    start += 1

new_cases_data = country_new_cases[start: start+30]  # <-- 30 days of data

#------------------------------------------------------#

I0 = country_total_cases[start]                    # <-- initial symptomatic value

days = range(len(new_cases_data))

X0 = [country_pop[country], 5*I0, I0, I0, 0, 0.01]

thry_aic = {}

# Fitting the model --------------------------------------------------------------------------#

for i in range(2):
    
    if i == 0 :
        
        # with environment ---------------------------------#

        init_params = [1.5] * 6  # <-- [bA, bI, bW, sA, sI, epsilon]
        bounds = ([0] * 6, [100, 100, 100, 100, 100, inc_pd*0.99])
        popt, pcov = curve_fit(I_WAIT, days, new_cases_data, p0=init_params, bounds=bounds)
        bA, bI, bW, sA, sI, epsilon = popt
        omega = 1/(inc_pd - epsilon)
        Y = odeint(dXdt, y0=X0, t=t, args=(bA, bI, bW, sA, sI, epsilon))
        new_cases_thry = daily_cases((1 - p) * omega * Y[:, 2])
        aic = AIC(new_cases_thry, new_cases_data, len(init_params))
        thry_aic['SEIR-W'] = (new_cases_thry, aic)

        #---------------------------------------------------#

    else:
        
        # without environment ------------------------------#

        init_params = [1.5] * 3  # <-- [bA, bI, epsilon]
        bounds = ([0] * 3, [100, 100, inc_pd*0.99])
        popt, pcov = curve_fit(I_noWAIT, days, new_cases_data, p0=init_params, bounds=bounds)
        bA, bI, epsilon = popt
        omega = 1/(inc_pd - epsilon)
        Y = odeint(dXdt, y0=X0, t=t, args=(bA, bI, 0, 0, 0, epsilon))
        new_cases_thry = daily_cases((1 - p) * omega * Y[:, 2])
        aic = AIC(new_cases_thry, new_cases_data, len(init_params))
        thry_aic['SEIR'] = (new_cases_thry, aic)

        #---------------------------------------------------#

#---------------------------------------------------------------------------------------------#

# Plotting the model --------------------------------------------------------------------------#

t = np.linspace(0, D, t_steps)  # Initializing the time over which to integrate (D sets the time scale)

fig, ax = plt.subplots(1, 2, figsize = [13, 4])
plt.subplots_adjust(wspace = 0.3)

max_y = max([thry_aic[x][0][-1] for x in thry_aic]+[new_cases_data[-1]])
delta_y = max_y - new_cases_data[0]

i = 0
for wait in thry_aic:
    
    new_cases_thry = thry_aic[wait][0]
    aic = thry_aic[wait][1]
    
    ax[i].plot(days, new_cases_thry, linewidth = 4, c = f'C{i}', zorder = 0)
    ax[i].scatter(days, new_cases_data, marker = 'x', linewidth = 3, s = 80, c = 'k', zorder = 1)
    ax[0].set_ylabel('Number of people', fontsize = 25)
    ax[i].set_title(f'{wait}', fontsize = 30, pad = 1)
    ax[i].tick_params(axis = 'both', direction = 'out', top = False, right = False, labelsize = 18, width = 3, length = 10)
    ax[i].ticklabel_format(axis = 'y', style = 'sci', scilimits = (0, 3))
    ax[i].yaxis.get_offset_text().set_fontsize(20)
    ax[i].yaxis.get_offset_text().set_position((-0.1, 10*max_y))
    ax[i].set_xlim(0, 30)
    ax[i].set_ylim(0, 1.5*1.06*max_y)
    ax[i].text(0.1, 0.45, f'AIC = {round(aic, 1)}', transform = ax[i].transAxes, fontsize = 30, fontweight = 'bold')
    ax[i].text(0.1, 0.80, f'{country}', transform = ax[i].transAxes, fontsize = 30, 
            bbox=dict(facecolor = 'none', edgecolor = 'k', boxstyle='round'), fontweight = 'bold')
#     ax[i].text(-0.3, 1.1, label[i], fontsize = 30, fontweight = 'bold', transform = ax[i].transAxes)
    ax[0].text(1, -0.3, 'Days', fontsize = 25, transform = ax[0].transAxes)
    ax[i].spines["top"].set_visible(False)
    ax[i].spines["right"].set_visible(False)
    ax[i].spines['left'].set_linewidth(3)
    ax[i].spines['bottom'].set_linewidth(3)
    
    i += 1
    
""" save to png with this line """
# plt.savefig(f'daily infected cases fit {country} (30 days)', dpi = 300, bbox_inches = 'tight')

#---------------------------------------------------------------------------------------------#


## PRCCs

### $\mathscr{R}_0$ PRCC

In [ ]:
""" running the R0 PRCC """

from pyDOE import *
from scipy.stats import uniform
import matplotlib as mpl
from numpy.linalg import inv

# mpl.rcParams['mathtext.fontset'] = 'stix'
# mpl.rcParams['font.family'] = 'STIXGeneral'

""" average parameter values """
bA = 0.5497938413115383
bI = 0.4909463148261044
bW = 0.031100446700197397
sA = 3.403914785083266
sI = 13.492166134223032
epsilon = 1/2.478385725999245  # <-- note the units of 1/days (instead of days)
omega = 0.33094892640811974
mu = 3.411863047817261*10**(-5)
muI = 0.0015851390386414379
nu = 0.03053968253968254
k = 0.6486486486486486
p = 0.956

# the dictionary below gives (mean,stdev) for each parameter
frac = 0.045  # <-- sets the range above and below which to sample
params = {'betaA': (bA, frac * bA),
         'betaI': (bI, frac * bI),
         'betaW': (bW, frac * bW),
         'sigmaA': (sA, frac * sA),
         'sigmaI': (sI, frac * sI),
         'epsln': (epsilon, frac * epsilon),
         'nu': (nu, frac * nu),
         'muI': (muI, frac * muI),
         'mu': (mu, frac * mu),
         'omeg': (omega, frac * omega),
         'k': (k, frac * k),
         'p': (p, frac * p)}

means = [x[0] for x in params.values()]
stdvs = [x[1] for x in params.values()]

# R0 PRCC

kk = len(params)
NN = 1000

Mu = (1 + NN)/2   # average rank

the_gammas = []

for s in range(50):
    
    """  - lhs is an NN x kk array of values between 0 and 1, sampled uniformly
         - lhd will map the lhs values across the cdf of the desired distribution (call it <dist>) 
            to the parameter spaces (the effect of .ppf())
         - we choose a uniform distribution (<dist> = uniform) so that strict parameter bounds are 
            maintained
         - lhd is a kk x NN array (confusingly) containing the parameter values 
    """
    
    lhd = lhs(kk, NN)   # <-- samples of percentages (sampled uniformly), lhd soon to be redefined

    for i in range(kk):  # this maps lhs choices via the desired dist. (uniform) to the parameter spaces
        """ uniform() takes params (left_value, sample_width) """
        lhd[:, i] = uniform(means[i] - stdvs[i], 2 * stdvs[i]).ppf(lhd[:, i])

    """ r gives the order-index of the parameters in lhd"""
    r = []
    for j in range(kk):
        x = lhd[:, j]
        seq = sorted(x)
        r.append([1 + seq.index(v) for v in x])

    r = np.array(r).T
    
    
    """ R0_lhd contains the resulting R0 values; R contains the order-indices"""
    R0_lhd = [Ro(*lhd[j,:]) for j in range(NN)]
    seq = sorted(R0_lhd)
    R = [1+seq.index(v) for v in R0_lhd]     

    """ C will contain correlations between order-vectors """
    C = np.zeros([kk + 1, kk + 1])    # initializes the matrix C

    """ computes correlations between parameters """
    for i in range(kk):
        for j in range(kk):
            rij = [(r[v,i] - Mu) * (r[v,j] - Mu) for v in range(NN)]
            rii = [(r[v,i] - Mu)**2 for v in range(NN)]
            rjj = [(r[v,j] - Mu)**2 for v in range(NN)]
            C[i,j] = sum(rij)/np.sqrt(sum(rii) * sum(rjj))

    """ computes correlations between the R0 values """
    for i in range(kk):
        rij = [(r[v,i] - Mu) * (R[v] - Mu) for v in range(NN)]
        rii = [(r[v,i] - Mu)**2 for v in range(NN)]
        rjj = [(R[v] - Mu)**2 for v in range(NN)]
        C[i,kk] = sum(rij)/np.sqrt(sum(rii) * sum(rjj))
        C[kk,i] = sum(rij)/np.sqrt(sum(rii) * sum(rjj))

    C[kk, kk] = 1
    
    
    """ the gammas are normalized """
    B = inv(C)
    
    gammas = [-B[i, kk]/np.sqrt(B[i, i] * B[kk, kk]) for i in range(kk)]

    the_gammas.append(gammas)

In [ ]:
""" plotting the R0 PRCC """

varbls = {'betaA': r'$\beta_A$',
         'betaI': r'$\beta_S$',
         'betaW': r'$\beta_W$',
         'sigmaA': r'$\sigma_A$',
         'sigmaI': r'$\sigma_S$',
         'epsln': r'$\epsilon$',
         'nu': r'$\nu$',
         'muI': r'$\mu_S$',
         'mu': r'$\mu$',
         'omeg': r'$\omega$',
         'k': '$k$',
         'p': '$p$'}


labels = varbls.values()
prcc_means = []
prcc_errs = []
for each in params:
    i = list(params.keys()).index(each)
    PRCCs = [the_gammas[j][i] for j in range(len(the_gammas))] 
    prcc_means.append(np.mean(PRCCs))
    prcc_errs.append(np.std(PRCCs))
    
fig, ax = plt.subplots(figsize = [10, 6])
ax.bar(labels, prcc_means, yerr=prcc_errs)
ax.plot(np.linspace(-1, len(params) + 1, 2), 0.5*np.ones(2), 'r--')
ax.plot(np.linspace(-1, len(params) + 1, 2), -0.5*np.ones(2), 'r--')
ax.plot(np.linspace(-1,len(params), 2), np.zeros(2), 'k')
plt.tick_params(labelsize=35, axis='x', direction='in',top=1,right=1)
plt.tick_params(labelsize=25, axis='y', direction='in',top=1,right=1)
ax.text(-0.1, 1.1, 'A', fontsize = 30, fontweight = 'bold', transform = ax.transAxes)

plt.title('$\mathscr{R}_0$ PRCC', fontsize=35, fontweight = 'bold')

plt.xlim(-1, len(params))
plt.ylim(-1.05, 1.05)
plt.tight_layout()

""" save to png with this line """
# plt.savefig('R0 PRCC', dpi = 300, bbox_inches = 'tight')

### 30 day total PRCC

In [ ]:
%%time 

from pyDOE import *
from scipy.stats import uniform
import matplotlib as mpl
from numpy.linalg import inv
                
def dXdt(X, t, ba, bi, bw, sa, si, eps, nuu, mui, muu, omeg, kappa, pp):
    S = X[0]
    E = X[1]
    A = X[2]
    I = X[3]
    R = X[4]
    W = X[5]

    N = S + E + A + I + R
    L = (ba * A + bi * I)/N + bw * W
    
#     omega = 1/(inc_pd - epsilon)
    
    dSdt = muu * (N - S) - L * S
    dEdt = L * S - (eps + muu) * E
    dAdt = eps * E - (omeg + muu) * A
    dIdt = (1 - pp) * omeg * A - (nuu + mui) * I
    dRdt = pp * omeg * A + nuu * I - muu * R
    dWdt = (sa * A + si * I) * (1 - W)/N - kappa * W

    return np.array([dSdt, dEdt, dAdt, dIdt, dRdt, dWdt])

def run_cumul_30(args):
    pp = args[11]
    omeg = args[9]
    Y = odeint(dXdt, y0=X0, t=t, args=tuple(args))
    return cumul_30((1 - pp) * omeg * Y[:, 2], a0 = X0[3])

""" average parameter values """
bA = 0.5497938413115383
bI = 0.4909463148261044
bW = 0.031100446700197397
sA = 3.403914785083266
sI = 13.492166134223032
epsilon = 1/2.478385725999245  # <-- note the units of 1/days (instead of days)
omega = 0.33094892640811974
mu = 3.411863047817261*10**(-5)
muI = 0.0015851390386414379
nu = 0.03053968253968254
k = 0.6486486486486486
p = 0.956

X0 = [57.05 * 10**6, 5*13.3, 13.3, 13.3, 0, 0.01]

# the dictionary below gives (mean,stdev) for each parameter
frac = 0.045  # <-- sets the range above and below which to sample
params = {'betaA': (bA, frac * bA),
         'betaI': (bI, frac * bI),
         'betaW': (bW, frac * bW),
         'sigmaA': (sA, frac * sA),
         'sigmaI': (sI, frac * sI),
         'epsln': (epsilon, frac * epsilon),
         'nu': (nu, frac * nu),
         'muI': (muI, frac * muI),
         'mu': (mu, frac * mu),
         'omeg': (omega, frac * omega),
         'k': (k, frac * k),
         'p': (p, frac * p)}

means = [x[0] for x in params.values()]
stdvs = [x[1] for x in params.values()]

kk = len(params)
NN = 1000

Mu = (1 + NN)/2   # average rank

the_gammas = []

for s in range(50):
    
    """  - lhs is an NN x kk array of values between 0 and 1, sampled uniformly
         - lhd will map the lhs values across the cdf of the desired distribution (call it <dist>) 
            to the parameter spaces (the effect of .ppf())
         - we choose a uniform distribution (<dist> = uniform) so that strict parameter bounds are 
            maintained
         - lhd is a kk x NN array (confusingly) containing the parameter values 
    """
    
    lhd = lhs(kk, NN)   # <-- samples of percentages (sampled uniformly), lhd soon to be redefined

    for i in range(kk):  # this maps lhs choices via the desired dist. (uniform) to the parameter spaces
        """ uniform() takes params (left_value, sample_width) """
        lhd[:, i] = uniform(means[i] - stdvs[i], 2 * stdvs[i]).ppf(lhd[:, i])

    """ r gives the order-index of the parameters in lhd"""
    r = []
    for j in range(kk):
        x = lhd[:, j]
        seq = sorted(x)
        r.append([1 + seq.index(v) for v in x])

    r = np.array(r).T
    
    
    """ cumul_30_lhd contains the resulting infected after 30 days value; R contains the order-indices"""
    cumul_30_lhd = [run_cumul_30(lhd[j,:]) for j in range(NN)]
    seq = sorted(cumul_30_lhd)
    R = [1+seq.index(v) for v in cumul_30_lhd]     

    """ C will contain correlations between order-vectors """
    C = np.zeros([kk + 1, kk + 1])    # initializes the matrix C

    """ computes correlations between parameters """
    for i in range(kk):
        for j in range(kk):
            rij = [(r[v,i] - Mu) * (r[v,j] - Mu) for v in range(NN)]
            rii = [(r[v,i] - Mu)**2 for v in range(NN)]
            rjj = [(r[v,j] - Mu)**2 for v in range(NN)]
            C[i,j] = sum(rij)/np.sqrt(sum(rii) * sum(rjj))

    """ computes correlations between the R0 values """
    for i in range(kk):
        rij = [(r[v,i] - Mu) * (R[v] - Mu) for v in range(NN)]
        rii = [(r[v,i] - Mu)**2 for v in range(NN)]
        rjj = [(R[v] - Mu)**2 for v in range(NN)]
        C[i,kk] = sum(rij)/np.sqrt(sum(rii) * sum(rjj))
        C[kk,i] = sum(rij)/np.sqrt(sum(rii) * sum(rjj))

    C[kk, kk] = 1
    
    
    """ the gammas are normalized """
    B = inv(C)
    
    gammas = [-B[i, kk]/np.sqrt(B[i, i] * B[kk, kk]) for i in range(kk)]

    the_gammas.append(gammas)

In [ ]:
""" plotting the 30 day total PRCC """

varbls = {'betaA': r'$\beta_A$',
         'betaI': r'$\beta_S$',
         'betaW': r'$\beta_W$',
         'sigmaA': r'$\sigma_A$',
         'sigmaI': r'$\sigma_S$',
         'epsln': r'$\epsilon$',
         'nu': r'$\nu$',
         'muI': r'$\mu_S$',
         'mu': r'$\mu$',
         'omeg': r'$\omega$',
         'k': '$k$',
         'p': '$p$'}


labels = varbls.values()
prcc_means = []
prcc_errs = []
for each in params:
    i = list(params.keys()).index(each)
    PRCCs = [the_gammas[j][i] for j in range(len(the_gammas))] 
    prcc_means.append(np.mean(PRCCs))
    prcc_errs.append(np.std(PRCCs))
    
fig, ax = plt.subplots(figsize = [10, 6])
ax.bar(labels, prcc_means, yerr=prcc_errs)
ax.plot(np.linspace(-1, len(params) + 1, 2), 0.5*np.ones(2), 'r--')
ax.plot(np.linspace(-1, len(params) + 1, 2), -0.5*np.ones(2), 'r--')
ax.text(-0.1, 1.1, 'B', fontsize = 30, fontweight = 'bold', transform = ax.transAxes)
ax.plot(np.linspace(-1,len(params), 2), np.zeros(2), 'k')
plt.tick_params(labelsize=35, axis='x', direction='in',top=1,right=1)
plt.tick_params(labelsize=25, axis='y', direction='in',top=1,right=1)
plt.title('30 day total PRCC', fontsize=35, fontweight = 'bold')
plt.xlim(-1, len(params))
plt.ylim(-1.05, 1.05)
plt.tight_layout()

""" save to png with this line """
# plt.savefig('infected after 30 days PRCC', dpi = 300, bbox_inches = 'tight')

### $t_{max}$ PRCC

In [ ]:
""" running the t_max PRCC """

from pyDOE import *
from scipy.stats import uniform
import matplotlib as mpl
from numpy.linalg import inv

def dXdt(X, t, ba, bi, bw, sa, si, eps, nuu, mui, muu, omeg, kappa, pp):
    S = X[0]
    E = X[1]
    A = X[2]
    I = X[3]
    R = X[4]
    W = X[5]

    N = S + E + A + I + R
    L = (ba * A + bi * I)/N + bw * W
    
#     omega = 1/(inc_pd - epsilon)
    
    dSdt = muu * (N - S) - L * S
    dEdt = L * S - (eps + muu) * E
    dAdt = eps * E - (omeg + muu) * A
    dIdt = (1 - pp) * omeg * A - (nuu + mui) * I
    dRdt = pp * omeg * A + nuu * I - muu * R
    dWdt = (sa * A + si * I) * (1 - W)/N - kappa * W

    return np.array([dSdt, dEdt, dAdt, dIdt, dRdt, dWdt])

def run_peaks(args):
    Y = odeint(dXdt, y0=X0, t=t, args=tuple(args))
    return peak(Y[:, 3])

""" average parameter values """
bA = 0.5497938413115383
bI = 0.4909463148261044
bW = 0.031100446700197397
sA = 3.403914785083266
sI = 13.492166134223032
epsilon = 1/2.478385725999245  # <-- note the units of 1/days (instead of days)
omega = 0.33094892640811974
mu = 3.411863047817261*10**(-5)
muI = 0.0015851390386414379
nu = 0.03053968253968254
k = 0.6486486486486486
p = 0.956

X0 = [57.05 * 10**6, 5*13.3, 13.3, 13.3, 0, 0.01]

# the dictionary below gives (mean,stdev) for each parameter
frac = 0.045  # <-- sets the range above and below which to sample
params = {'betaA': (bA, frac * bA),
         'betaI': (bI, frac * bI),
         'betaW': (bW, frac * bW),
         'sigmaA': (sA, frac * sA),
         'sigmaI': (sI, frac * sI),
         'epsln': (epsilon, frac * epsilon),
         'nu': (nu, frac * nu),
         'muI': (muI, frac * muI),
         'mu': (mu, frac * mu),
         'omeg': (omega, frac * omega),
         'k': (k, frac * k),
         'p': (p, frac * p)}

means = [x[0] for x in params.values()]
stdvs = [x[1] for x in params.values()]

kk = len(params)
NN = 1000

Mu = (1 + NN)/2   # average rank

the_gammas = []

for s in range(50):
    
    """  - lhs is an NN x kk array of values between 0 and 1, sampled uniformly
         - lhd will map the lhs values across the cdf of the desired distribution (call it <dist>) 
            to the parameter spaces (the effect of .ppf())
         - we choose a uniform distribution (<dist> = uniform) so that strict parameter bounds are 
            maintained
         - lhd is a kk x NN array (confusingly) containing the parameter values 
    """
    
    lhd = lhs(kk, NN)   # <-- samples of percentages (sampled uniformly), lhd soon to be redefined

    for i in range(kk):  # this maps lhs choices via the desired dist. (uniform) to the parameter spaces
        """ uniform() takes params (left_value, sample_width) """
        lhd[:, i] = uniform(means[i] - stdvs[i], 2 * stdvs[i]).ppf(lhd[:, i])

    """ r gives the order-index of the parameters in lhd"""
    r = []
    for j in range(kk):
        x = lhd[:, j]
        seq = sorted(x)
        r.append([1 + seq.index(v) for v in x])

    r = np.array(r).T
    
    
    """ peaks_lhd contains the resulting peak times; R contains the order-indices"""
    peaks_lhd = [run_peaks(lhd[j,:])[0] for j in range(NN)]
    seq = sorted(peaks_lhd)
    R = [1+seq.index(v) for v in peaks_lhd]     

    """ C will contain correlations between order-vectors """
    C = np.zeros([kk + 1, kk + 1])    # initializes the matrix C

    """ computes correlations between parameters """
    for i in range(kk):
        for j in range(kk):
            rij = [(r[v,i] - Mu) * (r[v,j] - Mu) for v in range(NN)]
            rii = [(r[v,i] - Mu)**2 for v in range(NN)]
            rjj = [(r[v,j] - Mu)**2 for v in range(NN)]
            C[i,j] = sum(rij)/np.sqrt(sum(rii) * sum(rjj))

    """ computes correlations between the R0 values """
    for i in range(kk):
        rij = [(r[v,i] - Mu) * (R[v] - Mu) for v in range(NN)]
        rii = [(r[v,i] - Mu)**2 for v in range(NN)]
        rjj = [(R[v] - Mu)**2 for v in range(NN)]
        C[i,kk] = sum(rij)/np.sqrt(sum(rii) * sum(rjj))
        C[kk,i] = sum(rij)/np.sqrt(sum(rii) * sum(rjj))

    C[kk, kk] = 1
    
    
    """ the gammas are normalized """
    B = inv(C)
    
    gammas = [-B[i, kk]/np.sqrt(B[i, i] * B[kk, kk]) for i in range(kk)]

    the_gammas.append(gammas)

In [ ]:
""" plotting the t_max PRCC """

varbls = {'betaA': r'$\beta_A$',
         'betaI': r'$\beta_S$',
         'betaW': r'$\beta_W$',
         'sigmaA': r'$\sigma_A$',
         'sigmaI': r'$\sigma_S$',
         'epsln': r'$\epsilon$',
         'nu': r'$\nu$',
         'muI': r'$\mu_S$',
         'mu': r'$\mu$',
         'omeg': r'$\omega$',
         'k': '$k$',
         'p': '$p$'}

labels = varbls.values()
prcc_means = []
prcc_errs = []
for each in params:
    i = list(params.keys()).index(each)
    PRCCs = [the_gammas[j][i] for j in range(len(the_gammas))] 
    prcc_means.append(np.mean(PRCCs))
    prcc_errs.append(np.std(PRCCs))
    
fig, ax = plt.subplots(figsize = [10, 6])
ax.bar(labels, prcc_means, yerr=prcc_errs)
ax.plot(np.linspace(-1, len(params) + 1, 2), 0.5*np.ones(2), 'r--')
ax.plot(np.linspace(-1, len(params) + 1, 2), -0.5*np.ones(2), 'r--')
ax.plot(np.linspace(-1,len(params), 2), np.zeros(2), 'k')

ax.text(-0.1, 1.1, 'C', fontsize = 30, fontweight = 'bold', transform = ax.transAxes)

plt.tick_params(labelsize=35, axis='x', direction='in',top=1,right=1)
plt.tick_params(labelsize=25, axis='y', direction='in',top=1,right=1)
plt.title('$t_{max}$ PRCC', fontsize=35, fontweight = 'bold')
plt.xlim(-1, len(params))
plt.ylim(-1.05, 1.05)
plt.tight_layout()

""" save to png with this line """
# plt.savefig('time to peaks PRCC', dpi = 300, bbox_inches = 'tight')

### $I_{max}$ PRCC

In [ ]:
""" running the I_max PRCC """

from pyDOE import *
from scipy.stats import uniform
import matplotlib as mpl
from numpy.linalg import inv

def dXdt(X, t, ba, bi, bw, sa, si, eps, nuu, mui, muu, omeg, kappa, pp):
    S = X[0]
    E = X[1]
    A = X[2]
    I = X[3]
    R = X[4]
    W = X[5]

    N = S + E + A + I + R
    L = (ba * A + bi * I)/N + bw * W
    
#     omega = 1/(inc_pd - epsilon)
    
    dSdt = muu * (N - S) - L * S
    dEdt = L * S - (eps + muu) * E
    dAdt = eps * E - (omeg + muu) * A
    dIdt = (1 - pp) * omeg * A - (nuu + mui) * I
    dRdt = pp * omeg * A + nuu * I - muu * R
    dWdt = (sa * A + si * I) * (1 - W)/N - kappa * W

    return np.array([dSdt, dEdt, dAdt, dIdt, dRdt, dWdt])

def run_peaks(args):
    Y = odeint(dXdt, y0=X0, t=t, args=tuple(args))
    return peak(Y[:, 3])

""" average parameter values """
bA = 0.5497938413115383
bI = 0.4909463148261044
bW = 0.031100446700197397
sA = 3.403914785083266
sI = 13.492166134223032
epsilon = 1/2.478385725999245  # <-- note the units of 1/days (instead of days)
omega = 0.33094892640811974
mu = 3.411863047817261*10**(-5)
muI = 0.0015851390386414379
nu = 0.03053968253968254
k = 0.6486486486486486
p = 0.956

X0 = [57.05 * 10**6, 5*13.3, 13.3, 13.3, 0, 0.01]

# the dictionary below gives (mean,stdev) for each parameter
frac = 0.045  # <-- sets the range above and below which to sample
params = {'betaA': (bA, frac * bA),
         'betaI': (bI, frac * bI),
         'betaW': (bW, frac * bW),
         'sigmaA': (sA, frac * sA),
         'sigmaI': (sI, frac * sI),
         'epsln': (epsilon, frac * epsilon),
         'nu': (nu, frac * nu),
         'muI': (muI, frac * muI),
         'mu': (mu, frac * mu),
         'omeg': (omega, frac * omega),
         'k': (k, frac * k),
         'p': (p, frac * p)}

means = [x[0] for x in params.values()]
stdvs = [x[1] for x in params.values()]

kk = len(params)
NN = 1000

Mu = (1 + NN)/2   # average rank

the_gammas = []

for s in range(50):
    
    """  - lhs is an NN x kk array of values between 0 and 1, sampled uniformly
         - lhd will map the lhs values across the cdf of the desired distribution (call it <dist>) 
            to the parameter spaces (the effect of .ppf())
         - we choose a uniform distribution (<dist> = uniform) so that strict parameter bounds are 
            maintained
         - lhd is a kk x NN array (confusingly) containing the parameter values 
    """
    
    lhd = lhs(kk, NN)   # <-- samples of percentages (sampled uniformly), lhd soon to be redefined

    for i in range(kk):  # this maps lhs choices via the desired dist. (uniform) to the parameter spaces
        """ uniform() takes params (left_value, sample_width) """
        lhd[:, i] = uniform(means[i] - stdvs[i], 2 * stdvs[i]).ppf(lhd[:, i])

    """ r gives the order-index of the parameters in lhd"""
    r = []
    for j in range(kk):
        x = lhd[:, j]
        seq = sorted(x)
        r.append([1 + seq.index(v) for v in x])

    r = np.array(r).T
    
    
    """ peaks_lhd contains the resulting peak values; R contains the order-indices"""
    peaks_lhd = [run_peaks(lhd[j,:])[1] for j in range(NN)]
    seq = sorted(peaks_lhd)
    R = [1+seq.index(v) for v in peaks_lhd]     

    """ C will contain correlations between order-vectors """
    C = np.zeros([kk + 1, kk + 1])    # initializes the matrix C

    """ computes correlations between parameters """
    for i in range(kk):
        for j in range(kk):
            rij = [(r[v,i] - Mu) * (r[v,j] - Mu) for v in range(NN)]
            rii = [(r[v,i] - Mu)**2 for v in range(NN)]
            rjj = [(r[v,j] - Mu)**2 for v in range(NN)]
            C[i,j] = sum(rij)/np.sqrt(sum(rii) * sum(rjj))

    """ computes correlations between the R0 values """
    for i in range(kk):
        rij = [(r[v,i] - Mu) * (R[v] - Mu) for v in range(NN)]
        rii = [(r[v,i] - Mu)**2 for v in range(NN)]
        rjj = [(R[v] - Mu)**2 for v in range(NN)]
        C[i,kk] = sum(rij)/np.sqrt(sum(rii) * sum(rjj))
        C[kk,i] = sum(rij)/np.sqrt(sum(rii) * sum(rjj))

    C[kk, kk] = 1
    
    
    """ the gammas are normalized """
    B = inv(C)
    
    gammas = [-B[i, kk]/np.sqrt(B[i, i] * B[kk, kk]) for i in range(kk)]

    the_gammas.append(gammas)

In [ ]:
""" plotting the I_max PRCC """

varbls = {'betaA': r'$\beta_A$',
         'betaI': r'$\beta_S$',
         'betaW': r'$\beta_W$',
         'sigmaA': r'$\sigma_A$',
         'sigmaI': r'$\sigma_S$',
         'epsln': r'$\epsilon$',
         'nu': r'$\nu$',
         'muI': r'$\mu_S$',
         'mu': r'$\mu$',
         'omeg': r'$\omega$',
         'k': '$k$',
         'p': '$p$'}


labels = varbls.values()
prcc_means = []
prcc_errs = []
for each in params:
    i = list(params.keys()).index(each)
    PRCCs = [the_gammas[j][i] for j in range(len(the_gammas))] 
    prcc_means.append(np.mean(PRCCs))
    prcc_errs.append(np.std(PRCCs))
    
fig, ax = plt.subplots(figsize = [10, 6])
ax.bar(labels, prcc_means, yerr=prcc_errs)
ax.plot(np.linspace(-1, len(params) + 1, 2), 0.5*np.ones(2), 'r--')
ax.plot(np.linspace(-1, len(params) + 1, 2), -0.5*np.ones(2), 'r--')
ax.plot(np.linspace(-1,len(params), 2), np.zeros(2), 'k')
plt.tick_params(labelsize=35, axis='x', direction='in',top=1,right=1)
plt.tick_params(labelsize=25, axis='y', direction='in',top=1,right=1)
plt.title('$I_{max}$ PRCC', fontsize=35, fontweight = 'bold')

ax.text(-0.1, 1.1, 'D', fontsize = 30, fontweight = 'bold', transform = ax.transAxes)

plt.xlim(-1, len(params))
plt.ylim(-1.05, 1.05)
plt.tight_layout()

""" save to png with this line """
# plt.savefig('infection peak PRCC', dpi = 300, bbox_inches = 'tight')

## Tornado subcomponents

In [ ]:
""" average parameters """
bA = 0.5497938413115383
bI = 0.4909463148261044
bW = 0.031100446700197397
sA = 3.403914785083266
sI = 13.492166134223032
epsilon = 1/2.478385725999245  # <-- note the use of 1/days here (instead of days)
omega = 0.33094892640811974
mu = 3.411863047817261*10**(-5)
muI = 0.0015851390386414379
nu = 0.03053968253968254
k = 0.6486486486486486
p = 0.956

frac = 0.045
dy = 0.0768

def Rp(ba, bi, bw, sa, si, eps, nuu, mui, muu, omeg, kk, pp):
    num = eps * (ba * (mui + nuu) + bi * (1 - pp) * omeg)
    den = (muu + eps) * (muu + omeg) * (mui + nuu)
    return num/den

def Re(ba, bi, bw, sa, si, eps, nuu, mui, muu, omeg, kk, pp):
    num = eps * bw * (sa * (mui + nuu) + si * (1 - pp) * omeg)
    den = kk * (muu + eps) * (muu + omeg) * (mui + nuu)
    return num/den

def simpleR0(ba, bi, bw, sa, si, eps, nuu, mui, muu, omeg, kk, pp):
    rp = Rp(ba, bi, bw, sa, si, eps, nuu, mui, muu, omeg, kk, pp)
    re = Re(ba, bi, bw, sa, si, eps, nuu, mui, muu, omeg, kk, pp)
    return (rp + np.sqrt(rp**2 + 4 * re**2))/2

""" we exclude p for these graphs """
varbls = {'omeg': (r'$\omega$', 9, 'k'),
          'betaA': (r'$\beta_A$', 0, 'b'),
          'betaI': (r'$\beta_S$', 1, 'b'),
          'nu': (r'$\nu$', 6, 'k'),
          'betaW': (r'$\beta_W$', 2, 'g'),
          'k': ('$k$', 10, 'g'),
          'sigmaA': (r'$\sigma_A$', 3, 'g'),
          'sigmaI': (r'$\sigma_S$', 4, 'g'),
          'muI': (r'$\mu_S$', 7, 'k'),
          'epsln': (r'$\epsilon$', 5, 'k'),
          'mu': (r'$\mu$', 8, 'k')
         }


""" choose the R0 function """

# for Rp ---------------------------------------------------------------#

# R0_function, R0_label, xloc, label = Rp, 'R_p', 0.36, 'A'
# xlow, xhigh = (2.2, 2.8)
# start = 0.81
# loc_descrip = {start - 0*dy:'$\leftarrow$ ppl-ppl asymp. transmission',
#                start - 1*dy:'$\leftarrow$ ppl-ppl symp. transmission',
#               }

#-----------------------------------------------------------------------#

# for Re ---------------------------------------------------------------#

R0_function, R0_label, xloc, label = Re, 'R_e^2', 0.42, 'B'
xlow, xhigh = (1.2, 1.8)
start = 0.785
loc_descrip = {start - 3*dy:'$\leftarrow$ env to ppl transmission',
               start - 4*dy:'$\leftarrow$ viral decay rate on surfaces',
               start - 5*dy:'$\leftarrow$ asymp. ppl to env transmission', 
               start - 6*dy:'$\leftarrow$ symp. ppl to env transmission'
              }

#-----------------------------------------------------------------------#

base = R0_function(bA, bI, bW, sA, sI, epsilon, nu, muI, mu, omega, k, p)

lows = []
highs = []

varz = [bA, bI, bW, sA, sI, epsilon, nu, muI, mu, omega, k, p]  # <-- in the order of the R0 formula args.

for var in varbls:
    
    new_varz = list(varz)
    i = varbls[var][1]
    
    # increment up:
    new_varz[i] = varz[i] * (1 + frac)
    highs.append(R0_function(*new_varz) - base)
    
    # increment down:
    new_varz[i] = varz[i] * (1 - frac)
    lows.append(R0_function(*new_varz) - base)


# The drawing part --------------------------------------------------------------------#

fig = plt.figure(figsize=[12, 8])

# The y position for each variable
ys = range(len(varbls))[::-1]  # top to bottom

# Plot the bars, one by one
for var, y, low, high in zip(varbls, ys, lows, highs):
    print(var, low + base, high + base)
    
    # Each bar is a "broken" horizontal bar chart
    if low < 0:
         # The width of the 'low' and 'high' pieces
        low_width = -low
        high_width = high
        plt.broken_barh(
        [(base + low, low_width), (base, high_width)],
        (y - 0.4, 0.8),
        facecolors=['white', 'black'],  # Try different colors if you like
        edgecolors=['black', 'black'],
        linewidth=1,
        )
    else:
        low_width = low
        high_width = -high
        plt.broken_barh(
        [(base, low_width), (base + high, high_width)],
        (y - 0.4, 0.8),
        facecolors=['white', 'black'],  # Try different colors if you like
        edgecolors=['black', 'black'],
        linewidth=1,
        )

    # Display the value as text. It should be positioned in the center of
    # the 'high' bar, except if there isn't any room there, then it should be
    # next to bar instead. Comment in if want numbers included
#     x = base + high_width / 2
#     if x <= base + 50:
#         x = base + high_width + 50
#     plt.text(x, y, str(value), va='center', ha='center')

# Draw a vertical line down the middle
plt.axvline(base, color='black')

# Position the x-axis on the top, hide all the other spines (=axis lines)
axes = plt.gca()  # (gca = get current axes)
axes.spines['left'].set_visible(False)
axes.spines['right'].set_visible(False)
axes.spines['bottom'].set_visible(False)
axes.xaxis.set_ticks_position('top')

for loc in loc_descrip:
    axes.text(xloc, loc, loc_descrip[loc], transform = axes.transAxes, fontsize = 25)


# Make the y-axis display the variables
plt.yticks(ys, [x[0] for x in varbls.values()], fontsize = 35)

plt.xticks(fontsize = 30)

# title
# plt.suptitle('$R_0$ Uncertainty',fontsize=25,fontname='Times New Roman')

# Set the portion of the x- and y-axes to show
plt.xlim(xlow, xhigh)
plt.ylim(-1, len(varbls))

plt.title(f'${R0_label}$ single parameter sensitivity', fontsize = 35, pad = 30, fontweight = 'bold')

axes.text(-0.1, 1.2, label, fontsize = 30, fontweight = 'bold', transform = axes.transAxes)

colors = [x[2] for x in varbls.values()]
for ticklabel, tickcolor in zip(plt.gca().get_yticklabels(), colors):
    ticklabel.set_color(tickcolor)

# p.tight_layout()

""" save to png with this line """
# plt.savefig(f'{R0_label} tornado (no p)', dpi = 300, bbox_inches = 'tight')

In [ ]:
""" average parameters """
bA = 0.5497938413115383
bI = 0.4909463148261044
bW = 0.031100446700197397
sA = 3.403914785083266
sI = 13.492166134223032
epsilon = 1/2.478385725999245  # <-- note the use of 1/days here (instead of days)
omega = 0.33094892640811974
mu = 3.411863047817261*10**(-5)
muI = 0.0015851390386414379
nu = 0.03053968253968254
k = 0.6486486486486486
p = 0.956

frac = 0.045
start = 0.81
dy = 0.081

def Rp(ba, bi, bw, sa, si, eps, nuu, mui, muu, omeg, kk, pp):
    num = eps * (ba * (mui + nuu) + bi * (1 - pp) * omeg)
    den = (muu + eps) * (muu + omeg) * (mui + nuu)
    return num/den

def Re(ba, bi, bw, sa, si, eps, nuu, mui, muu, omeg, kk, pp):
    num = eps * bw * (sa * (mui + nuu) + si * (1 - pp) * omeg)
    den = kk * (muu + eps) * (muu + omeg) * (mui + nuu)
    return num/den

def simpleR0(ba, bi, bw, sa, si, eps, nuu, mui, muu, omeg, kk, pp):
    rp = Rp(ba, bi, bw, sa, si, eps, nuu, mui, muu, omeg, kk, pp)
    re = Re(ba, bi, bw, sa, si, eps, nuu, mui, muu, omeg, kk, pp)
    return (rp + np.sqrt(rp**2 + 4 * re))/2

""" we exclude p for these graphs """
varbls = {'omeg': (r'$\omega$', 9, 'k'),
          'betaA': (r'$\beta_A$', 0, 'b'),
          'betaI': (r'$\beta_S$', 1, 'b'),
          'nu': (r'$\nu$', 6, 'k'),
          'betaW': (r'$\beta_W$', 2, 'g'),
          'k': ('$k$', 10, 'g'),
          'sigmaA': (r'$\sigma_A$', 3, 'g'),
          'sigmaI': (r'$\sigma_S$', 4, 'g'),
          'muI': (r'$\mu_S$', 7, 'k'),
          'epsln': (r'$\epsilon$', 5, 'k'),
          'mu': (r'$\mu$', 8, 'k')
         }


""" choose the R0 function """

# for Rp ---------------------------------------------------------------#

R0_function, R0_label, xloc, label = Rp, 'R_p', 0.36, 'A'
xlow, xhigh = (2.2, 2.8)
start = 0.81
loc_descrip = {start - 0*dy:'$\leftarrow$ ppl-ppl asymp. transmission',
               start - 1*dy:'$\leftarrow$ ppl-ppl symp. transmission',
              }

#-----------------------------------------------------------------------#

# for Re ---------------------------------------------------------------#

# R0_function, R0_label, xloc, label = Re, 'R_e^2', 0.42, 'B'
# xlow, xhigh = (1.2, 1.8)
# loc_descrip = {start - 3*dy:'$\leftarrow$ env to ppl transmission',
#                start - 4*dy:'$\leftarrow$ viral decay rate on surfaces',
#                start - 5*dy:'$\leftarrow$ asymp. ppl to env transmission', 
#                start - 6*dy:'$\leftarrow$ symp. ppl to env transmission'
#               }

#-----------------------------------------------------------------------#

# for R0 ---------------------------------------------------------------#

# R0_function, R0_label, xloc, label = simpleR0, '\mathscr{R}_0', 0.32, 'C'
# xlow, xhigh = (2.7, 3.3)
# loc_descrip = {start - 0*dy:'$\leftarrow$ ppl-ppl asymp. transmission',
#                start - 1*dy:'$\leftarrow$ ppl-ppl symp. transmission',
#                start - 3*dy:'$\leftarrow$ env to ppl transmission',
#                start - 4*dy:'$\leftarrow$ viral decay rate on surfaces',
#                start - 5*dy:'$\leftarrow$ asymp. ppl to env transmission', 
#                start - 6*dy:'$\leftarrow$ symp. ppl to env transmission'
#               }

#-----------------------------------------------------------------------#

base = R0_function(bA, bI, bW, sA, sI, epsilon, nu, muI, mu, omega, k, p)

lows = []
highs = []

varz = [bA, bI, bW, sA, sI, epsilon, nu, muI, mu, omega, k, p]  # <-- in the order of the R0 formula args.

for var in varbls:
    
    new_varz = list(varz)
    i = varbls[var][1]
    
    # increment up:
    new_varz[i] = varz[i] * (1 + frac)
    highs.append(R0_function(*new_varz) - base)
    
    # increment down:
    new_varz[i] = varz[i] * (1 - frac)
    lows.append(R0_function(*new_varz) - base)


# The drawing part --------------------------------------------------------------------#

fig = plt.figure(figsize=[12, 8])

# The y position for each variable
ys = range(len(varbls))[::-1]  # top to bottom

# Plot the bars, one by one
for var, y, low, high in zip(varbls, ys, lows, highs):
    print(var, low + base, high + base)
    
    # Each bar is a "broken" horizontal bar chart
    if low < 0:
         # The width of the 'low' and 'high' pieces
        low_width = -low
        high_width = high
        plt.broken_barh(
        [(base + low, low_width), (base, high_width)],
        (y - 0.4, 0.8),
        facecolors=['white', 'black'],  # Try different colors if you like
        edgecolors=['black', 'black'],
        linewidth=1,
        )
    else:
        low_width = low
        high_width = -high
        plt.broken_barh(
        [(base, low_width), (base + high, high_width)],
        (y - 0.4, 0.8),
        facecolors=['white', 'black'],  # Try different colors if you like
        edgecolors=['black', 'black'],
        linewidth=1,
        )

    # Display the value as text. It should be positioned in the center of
    # the 'high' bar, except if there isn't any room there, then it should be
    # next to bar instead. Comment in if want numbers included
#     x = base + high_width / 2
#     if x <= base + 50:
#         x = base + high_width + 50
#     plt.text(x, y, str(value), va='center', ha='center')

# Draw a vertical line down the middle
plt.axvline(base, color='black')

# Position the x-axis on the top, hide all the other spines (=axis lines)
axes = plt.gca()  # (gca = get current axes)
axes.spines['left'].set_visible(False)
axes.spines['right'].set_visible(False)
axes.spines['bottom'].set_visible(False)
axes.xaxis.set_ticks_position('top')

for loc in loc_descrip:
    axes.text(xloc, loc, loc_descrip[loc], transform = axes.transAxes, fontsize = 25)


# Make the y-axis display the variables
plt.yticks(ys, [x[0] for x in varbls.values()], fontsize = 35)

plt.xticks(fontsize = 30)

# title
# plt.suptitle('$R_0$ Uncertainty',fontsize=25,fontname='Times New Roman')

# Set the portion of the x- and y-axes to show
plt.xlim(xlow, xhigh)
plt.ylim(-1, len(varbls))

plt.title(f'${R0_label}$ single parameter sensitivity', fontsize = 35, pad = 30, fontweight = 'bold')

axes.text(-0.1, 1.2, label, fontsize = 30, fontweight = 'bold', transform = axes.transAxes)

colors = [x[2] for x in varbls.values()]
for ticklabel, tickcolor in zip(plt.gca().get_yticklabels(), colors):
    ticklabel.set_color(tickcolor)

# p.tight_layout()

""" save to png with this line """
# plt.savefig(f'{R0_label} tornado (no p)', dpi = 300, bbox_inches = 'tight')

## "Combinatorics"

In [ ]:
""" RUN THIS CELL FIRST """

trial = 9

SA = load_obj(f'SA_{trial}')
SI = load_obj(f'SI_{trial}')
SAI = load_obj(f'SAI_{trial}')
SWA = load_obj(f'SWA_{trial}')
SWI = load_obj(f'SWI_{trial}')
SWAI = load_obj(f'SWAI_{trial}')
SAISWAI = load_obj(f'SAISWAI_{trial}')

letters = ['E', 'A', 'I', 'W']
combinations = [
            ('SA', SA),
            ('SI', SI),
            ('SAI', SAI),
            ('SWA', SWA),
            ('SWI', SWI),
            ('SWAI', SWAI),
            ('SAISWAI', SAISWAI)
]

descriptions = [
    'no env. infection; only $I_A$ infectious',
    'no env. infection; only $I_S$ infectious',
    'no env. infection; $I_A$ & $I_S$ infectious',
    'no ppl-ppl infection; only $I_A$ transmits to env.',
    'no ppl-ppl infection; only $I_S$ transmits to env.',
    'no ppl-ppl infection; $I_A$ & $I_S$ transmit to env.',
    r'env. & ppl-ppl infection; only $I_A$ & $I_S$ $\rightarrow$ env. + ppl'    
]

time_peak = {x[0]: {} for x in combinations}
time_peak_var = {x[0]: {} for x in combinations}

for label, array in combinations:
    for i, letter in zip(range(4), letters):
        """ (x, y) = (avg. time to peak, avg. peak value) """
        time_peak[label][letter] = (np.mean([x[i][0] for x in array]), np.mean([x[i][1] for x in array]))
        time_peak_var[label][letter] = (np.std([x[i][0] for x in array]), np.std([x[i][1] for x in array]))

    time_peak[label]['30'] = np.mean([x[-1] for x in array])
    time_peak_var[label]['30'] = np.std([x[-1] for x in array])


### $t_{max}$ combinatorics

In [ ]:
""" time to peaks """

import string

alphabet = string.ascii_uppercase


# set width of bar
barWidth = 0.2
 
# set height of bar
 
# times first --#
bars1 = [time_peak[comb[0]]['E'][0] for comb in combinations]
bars2 = [time_peak[comb[0]]['A'][0] for comb in combinations]
bars3 = [time_peak[comb[0]]['I'][0] for comb in combinations]
bars4 = [time_peak[comb[0]]['W'][0] for comb in combinations]

# errorbars
errs1 = [time_peak_var[comb[0]]['E'][0] for comb in combinations]
errs2 = [time_peak_var[comb[0]]['A'][0] for comb in combinations]
errs3 = [time_peak_var[comb[0]]['I'][0] for comb in combinations]
errs4 = [time_peak_var[comb[0]]['W'][0] for comb in combinations]
    
# Set position of bar on X axis
r1 = np.arange(len(bars1))
r2 = [x + barWidth for x in r1]
r3 = [x + barWidth for x in r2]
r4 = [x + barWidth for x in r3]
 
fig, ax = plt.subplots(figsize = [10, 6])    
    
# Make the plot
ax.bar(r1, bars1, color='#7f6d5f', width=barWidth, edgecolor='white', label='$E$', yerr=errs1)
ax.bar(r2, bars2, color='#557f2d', width=barWidth, edgecolor='white', label='$I_s$', yerr=errs2)
ax.bar(r3, bars3, color='#2d7f5e', width=barWidth, edgecolor='white', label='$I_c$', yerr=errs3)
ax.bar(r4, bars4, color='#d0f0c0', width=barWidth, edgecolor='white', label='$W$', yerr=errs4)
 
ax.set_title('$t_{max}$ (average parameters)', fontsize = 25)
plt.xticks([r + 1.5 * barWidth for r in range(len(bars1))], list(alphabet)[:len(bars1)])
ax.tick_params(direction = 'in', labelsize = 15)

ax.set_yscale('log') 
ax.set_xlabel('combination', fontsize = 15)
ax.set_ylabel('time / days', fontsize = 15)

label_descrip = r''
for each, letter in zip(descriptions, list(alphabet[:len(bars1)])):
    label_descrip = label_descrip + letter + ': ' + each + '\n'
    
ax.text(1.01, 0.25, label_descrip, transform = ax.transAxes, fontsize = 15)

# Create legend & Show graphic
ax.legend(fontsize = 15, loc = (1.01, 0.75))


""" save to png with this line """
# plt.savefig(f'time to peaks', dpi = 300, bbox_inches = 'tight')

### $I_{max}$ combinatorics

In [ ]:
""" peak values """

import string

alphabet = string.ascii_uppercase


# set width of bar
barWidth = 0.2
 
# set height of bar
 
# peaks next --#
bars1 = [time_peak[comb[0]]['E'][1] for comb in combinations]
bars2 = [time_peak[comb[0]]['A'][1] for comb in combinations]
bars3 = [time_peak[comb[0]]['I'][1] for comb in combinations]
bars4 = [time_peak[comb[0]]['W'][1] for comb in combinations]

# errorbars
errs1 = [time_peak_var[comb[0]]['E'][1] for comb in combinations]
errs2 = [time_peak_var[comb[0]]['A'][1] for comb in combinations]
errs3 = [time_peak_var[comb[0]]['I'][1] for comb in combinations]
errs4 = [time_peak_var[comb[0]]['W'][1] for comb in combinations]
    
# Set position of bar on X axis
r1 = np.arange(len(bars1))
r2 = [x + barWidth for x in r1]
r3 = [x + barWidth for x in r2]
r4 = [x + barWidth for x in r3]
 
fig, ax = plt.subplots(figsize = [10, 6])    
    
# Make the plot
ax.bar(r1, bars1, color='#7f6d5f', width=barWidth, edgecolor='white', label='$E$', yerr=errs1)
ax.bar(r2, bars2, color='#557f2d', width=barWidth, edgecolor='white', label='$I_s$', yerr=errs2)
ax.bar(r3, bars3, color='#2d7f5e', width=barWidth, edgecolor='white', label='$I_c$', yerr=errs3)
ax.bar(r4, bars4, color='#d0f0c0', width=barWidth, edgecolor='white', label='$W$', yerr=errs4)
 
ax.set_title('Infection peaks (average parameters)', fontsize = 25)
plt.xticks([r + 1.5 * barWidth for r in range(len(bars1))], list(alphabet)[:len(bars1)])
ax.tick_params(direction = 'in', labelsize = 15)

label_descrip = ''
for each, letter in zip(descriptions, list(alphabet[:len(bars1)])):
    label_descrip = label_descrip + letter + ': ' + each + '\n'
    
ax.text(1.01, 0.25, label_descrip, transform = ax.transAxes, fontsize = 15)

ax.set_yscale('log') 
ax.set_xlabel('combination', fontsize = 15)
ax.set_ylabel('number of people', fontsize = 15)
    
# Create legend & Show graphic
ax.legend(fontsize = 15, loc = (1.01, 0.75))

""" save to png with this line """
# plt.savefig(f'peak values', dpi = 300, bbox_inches = 'tight')

### 30 day cumulative combinatorics

In [ ]:
""" number infected after 30 days """

def autolabel(rects):
    """
    Attach a text label above each bar displaying its height
    """
    for rect in rects:
        height = rect.get_height()
        ax.text(rect.get_x() + rect.get_width()/2 - 0.35, 1.05*height,
                '%d' % int(height),
                ha='center', va='bottom', fontsize = 15)

# set width of bar
barWidth = 0.80
 
# set height of bar
 
# peaks next --#
bars1 = [time_peak[comb[0]]['30'] for comb in combinations]

# errorbars
errs1 = [time_peak_var[comb[0]]['30'] for comb in combinations]
    
# Set position of bar on X axis
r1 = np.arange(len(bars1))
 
fig, ax = plt.subplots(figsize = [10, 6])    
    
# Make the plot
bars = ax.bar(r1, bars1, color='#d45b12', width=barWidth, edgecolor='white', yerr=errs1)

autolabel(bars)

ax.set_title('Total infected after 30 days', fontsize = 25)
plt.xticks(range(len(bars1)), list(alphabet)[:len(bars1)])
ax.tick_params(direction = 'in', labelsize = 15)

label_descrip = ''
for each, letter in zip(descriptions, list(alphabet[:len(bars1)])):
    label_descrip = label_descrip + letter + ': ' + each + '\n'
    
ax.text(1.01, 0.25, label_descrip, transform = ax.transAxes, fontsize = 15)

ax.set_yscale('log') 
ax.set_xlabel('combination', fontsize = 15)
ax.set_ylabel('number of people', fontsize = 15)
    
""" save to png with this line """
# plt.savefig(f'infected after 30 days', dpi = 300, bbox_inches = 'tight')